In [ ]:
## 1- Intalar a biblioteca google-genai  e google-adk##
%pip -q install google-genai
%pip -q install google-adk

In [ ]:
# 2- Configurar a API do google-genai
import os
from google.colab import userdata #importando a biblioteca userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY') # Defina a chave da API do Google GenAI

In [ ]:
# 3- Configurar cliente da SDK do Gemini
from google import genai #importando a biblioteca genai

client = genai.Client() # Configurando o cliente
MODEL_ID = "gemini-2.0-flash" # Definindo o modelo

In [ ]:
# 4- Importar bibliotecas do ADK e utilitários

from google.adk.agents import Agent #importando a biblioteca agent
from google.adk.runners import Runner #importando a biblioteca runner
from google.adk.sessions import InMemorySessionService #importando a biblioteca session
from google.adk.tools import google_search #importando a biblioteca google_search
from google.genai import types #importando a biblioteca types
from datetime import date #importando a biblioteca date
import textwrap #importando a biblioteca textwrap
import uuid # importando a biblioteca uuid
import nest_asyncio #importando a biblioteca nest_asyncio
import asyncio #importando a biblioteca asyncio
import warnings #importando a biblioteca warnings
from IPython.display import display, Markdown #importando a biblioteca display
import ipywidgets as widgets #importando a biblioteca widgets

warnings.filterwarnings("ignore") #ignorando os avisos
nest_asyncio.apply() #aplicando o nest_asyncio

In [ ]:
# 5- Funções auxiliares

def call_agent(agent: Agent, message_text: str) -> str: #função para chamar o agente
    session_service = InMemorySessionService() # Criando um serviço de sessão em memória
    session = asyncio.run(session_service.create_session(app_name=agent.name, user_id="user1")) # Criando uma sessão
    session_id = str(uuid.uuid4()) # Gerando um ID de sessão único
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service) # Criando um executor
    content = types.Content(role="user", parts=[types.Part(text=message_text)]) # Criando o conteúdo da mensagem

    final_response = ""  # Variável para armazenar a resposta final
    for event in asyncio.run(runner.run(user_id="user1", session_id=session_id, new_message=content)): # Executando o executor
        if event.is_final_response(): # Verificando se a resposta é final
            for part in event.content.parts: # Iterando sobre as partes da resposta
                if part.text: # Verificando se o texto da parte não está vazio
                    final_response += part.text + "\n" # Adicionando o texto da parte à resposta final
    return final_response # Retorna a resposta final

def to_markdown(text): # Função para converter texto em Markdown
    text = text.replace('•', '  *') # Substitui os marcadores de lista
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True)) # Adiciona um marcador de citação

In [ ]:
# 6 - Agentes

# Criando um agente de diagnóstico
agente_diagnostico = Agent( 
    name="Diagnóstico",
    description="Agente que faz o diagnóstico de um problema.",
    model=MODEL_ID,
    instruction="""
Você é um consultor experiente em gestão empresarial. Sua tarefa é interpretar o texto enviado por um gestor que descreve um desafio real em seu trabalho.

Extraia e retorne de forma clara:
- Tipo de desafio (ex: produtividade, comunicação, priorização, sobrecarga, etc.)
- Área afetada (ex: projetos, processos, equipe, tecnologia, etc.)
- Grau de urgência (baixa, média ou alta)

Texto do gestor:
"""
)

#Criando um agente de recomndação
agente_recomendacao = Agent(
    name="Recomendação",
    description="Agente que faz recomendações para resolver um problema.",
    model=MODEL_ID,
    instruction="""
Com base no diagnóstico a seguir, recomende 3 ferramentas ou métodos ou frameworks que sejam práticos para o gestor utilizar.
Você deve considerar o tipo de desafio, a área afetada e o grau de urgência.

Para cada recomendação, forneça:
- Tipo de desafio:
- Nome da ferramenta ou método:
- Descrição:
- Saídas esperadas:
- Complexidade de implementação: (baixa, média ou alta)

Diagnóstico:
{{input}}
"""
)

#criando um agente de Justificativa
agente_justificativa = Agent(
    name="Justificativa",
    description="Agente que cria a saída para resolver um problema.",
    model=MODEL_ID,
    tools=[google_search],
    instruction="""
Você é um consultor especializado em gestão organizacional. Com base no diagnóstico a seguir, forneça **3 recomendações** de ferramentas ou métodos que possam auxiliar o gestor, organizadas por ordem crescente de complexidade de implementação.

Para cada recomendação, apresente:
- Nome da ferramenta ou método e autor:
- Descrição:
- Tipo de desafio:
- Complexidade de implementação:
- Link para aprofundamento:

Diagnóstico:
{{input}}
"""
)



In [ ]:
# 7 - Função de orquestração dos agentes

def orquestrador(Texto_desafio): # Função para orquestrar os agentes
    resposta_diagnostico = call_agent(agente_diagnostico, Texto_desafio) # Chama o agente de diagnóstico
    display(to_markdown(f"## 🔍 Diagnóstico: \n{resposta_diagnostico}")) # Exibe a resposta do diagnóstico  
    resposta_recomendacao = call_agent(agente_recomendacao, resposta_diagnostico) # Chama o agente de recomendação
    display(to_markdown(f"## ✅ Recomendações: \n{resposta_recomendacao}")) # Exibe a resposta da recomendação
   
    entrada_justificativa = f"Diagnóstico: \n {resposta_diagnostico.strip()} \nRecomendações: {resposta_recomendacao.strip()}" # Cria a entrada para o agente de justificativa
    resposta_justificativa = call_agent(agente_justificativa, entrada_justificativa) # Chama o agente de justificativa
    display(to_markdown(f"## 📝 Justificativa: \n{resposta_justificativa}")) # Exibe a resposta da justificativa

In [ ]:
# 8- Interface Gráfica

entrada_usuario = widgets.Textarea(
    description="Desafio:",
    placeholder="Descreva o desafio enfrentado....\n Forneça informações como área, porte da empresa, quantidade de membros da equipe, etc.",
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': 'initial'}
) # Criando uma área de texto para a entrada do usuário

botao_enviar = widgets.Button(
    description="Enviar",
    button_style='success',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'},
    tooltip="Clique para enviar o desafio"
) # Criando um botão para enviar a entrada

def ao_clicar_botao(b): # Função para lidar com o clique do botão
    if not texto_desafio.strip():
        display(Markdown("### ⚠️ Por favor, descreva o desafio antes de enviar."))
    else:
        orquestrador(texto_desafio) # Chama a função orquestrador com o texto do desafio
        entrada_usuario.value = "" # Limpa a entrada do usuário
botao_enviar.on_click(ao_clicar_botao) # Vincula a função ao clique do botão
display(entrada_usuario, botao_enviar) # Exibe a interface gráfica